## 1) Install dependencies

In [3]:
!pip install docplex geopy folium cplex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.5/633.5 kB 7.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for docplex: filename=docplex-2.25.236-py3-none-any.whl size=671351 sha256=06b08bed08b8e5e034a60b5ca19d62fc31830d065a293495cc620344cf1e1a87
  Stored in directory: /root/.cache/pip/wheels/3b/e5/00/0bf0173d67188fe73a13e3a61412b3f975f60205e3fab93a69
Successfully built docplex


### Step 1: Import the packages

In [15]:
import sys
import docplex.mp
import geopy 
import folium
import cplex

### Step 2: Model the data

* The data for this problem is quite simple: it is composed of the list of public libraries and their geographical locations.
* The data is acquired from Chicago open data as a JSON file, which is in the following format:

`data" : [ [ 1, "13BFA4C7-78CE-4D83-B53D-B57C60B701CF", 1, 1441918880, "885709", 1441918880, "885709", null, "Albany Park", "M, W: 10AM-6PM;  TU, TH: 12PM-8PM; F, SA: 9AM-5PM; SU: Closed", "Yes", "Yes ", "3401 W. Foster Avenue", "CHICAGO", "IL", "60625", "(773) 539-5450", [ "http://www.chipublib.org/locations/1/", null ], [ null, "41.975456", "-87.71409", null, false ] ]
This code snippet represents library "**3401 W. Foster Avenue**" located at **41.975456, -87.71409**`

### Step 3: Prepare the data
We need to collect the list of public libraries locations and keep their names, latitudes, and longitudes.

In [6]:
# Store longitude, latitude and street crossing name of each public library location.
class XPoint(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __str__(self):
        return "P(%g_%g)" % (self.x, self.y)

class NamedPoint(XPoint):
    def __init__(self, name, x, y):
        XPoint.__init__(self, x, y)
        self.name = name
    def __str__(self):
        return self.name

In [7]:
# Defining how to compute the earth distance between 2 points
from geopy.distance import great_circle
 
def get_distance(p1, p2):
    return great_circle((p1.y, p1.x), (p2.y, p2.x)).miles

### Declare the list of libraries
Parse the JSON file to get the list of libraries and store them as Python elements

In [8]:
# Function to build public libraries in the data format that we need
def build_libraries_from_url(url, name_pos, lat_long_pos):
    import requests
    import json

    r = requests.get(url)
    myjson = json.loads(r.text, parse_constant='utf-8')
    myjson = myjson['data']

    libraries = []
    k = 1
    for location in myjson:
        uname = location[name_pos]
        try:
            latitude = float(location[lat_long_pos][1])
            longitude = float(location[lat_long_pos][2])
        except TypeError:
            latitude = longitude = None
        try:
            name = str(uname)
        except:
            name = "???"
        name = "P_%s_%d" % (name, k)
        if latitude and longitude:
            cp = NamedPoint(name, longitude, latitude)
            libraries.append(cp)
            k += 1
    return libraries

In [9]:
libraries = build_libraries_from_url('https://data.cityofchicago.org/api/views/x8fc-8rcq/rows.json?accessType=DOWNLOAD',
                                   name_pos=10,
                                   lat_long_pos=16)

In [10]:
print("There are %d public libraries in Chicago" % (len(libraries)))

There are 81 public libraries in Chicago


In [11]:
# Define the number of shops we need to open 
nb_shops = 5
print("We would like to open %d coffee shops" % nb_shops)

We would like to open 5 coffee shops


In [12]:
# Display the library locations on map
import folium
map_osm = folium.Map(location=[41.878, -87.629], zoom_start=11)
for library in libraries:
    lt = library.y
    lg = library.x
    folium.Marker([lt, lg]).add_to(map_osm)
map_osm

# 2) Model building

### Step 1: Set up the prescriptive model

In [39]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

* system is: Linux 64bit
* Python version 3.10.13, located at: /opt/conda/bin/python3.10
* docplex is present, version is 2.25.236
* CPLEX library is present, version is 22.1.1.0, located at: /opt/conda/lib/python3.10/site-packages
* pandas is present, version is 2.2.1


In [40]:
# Create the docplex model
# This model contains all the business constraints and defines the objective.

from docplex.mp.model import Model

mdl = Model("chicago coffee shops")

In [41]:
# Define the decision variables
import random

BIGNUM = 999999999

# Ensure unique points
selected_locations = list(libraries)[:20]
libraries = set(selected_locations)
# For simplicity, let's consider that coffee shops candidate locations are the same as libraries locations.
# That is: any library location can also be selected as a coffee shop.
coffeeshop_locations = libraries
print(len(coffeeshop_locations))

# Decision vars
# Binary vars indicating which coffee shop locations will be actually selected
coffeeshop_vars = mdl.binary_var_dict(coffeeshop_locations, name="is_coffeeshop")
#
# Binary vars representing the "assigned" libraries for each coffee shop
link_vars = mdl.binary_var_matrix(coffeeshop_locations, libraries, "link")

20


### Step 2: Express the business constraints

In [42]:
# First constraint - If the distance is suspect it needs to be excluded from the problem
for c_loc in coffeeshop_locations:
    for b in libraries:
        if get_distance(c_loc, b) >= BIGNUM:
            mdl.add_constraint(link_vars[c_loc, b] == 0, "ct_forbid_{0!s}_{1!s}".format(c_loc, b))

In [43]:
# Second constraint: Each library must be linked to a coffee shop that is open.
mdl.add_constraints(link_vars[c_loc, b] <= coffeeshop_vars[c_loc]
                   for b in libraries
                   for c_loc in coffeeshop_locations)
mdl.print_information()

Model: chicago coffee shops
 - number of variables: 420
   - binary=420, integer=0, continuous=0
 - number of constraints: 400
   - linear=400
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [44]:
# Third constraint: Each library is linked to exactly one coffee shop.
mdl.add_constraints(mdl.sum(link_vars[c_loc, b] for c_loc in coffeeshop_locations) == 1
                   for b in libraries)
mdl.print_information()

Model: chicago coffee shops
 - number of variables: 420
   - binary=420, integer=0, continuous=0
 - number of constraints: 420
   - linear=420
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [45]:
# Fourth constraint: there is a fixed number of coffee shops to open.
# Total nb of open coffee shops
mdl.add_constraint(mdl.sum(coffeeshop_vars[c_loc] for c_loc in coffeeshop_locations) == nb_shops)

# Print model information
mdl.print_information()

Model: chicago coffee shops
 - number of variables: 420
   - binary=420, integer=0, continuous=0
 - number of constraints: 421
   - linear=421
 - parameters: defaults
 - objective: none
 - problem type is: MILP


### Step 3: Express the objective

The objective is to minimize the total distance from libraries to coffee shops so that a book reader always gets to our coffee shop easily.

In [46]:
# Minimize total distance from points to hubs
total_distance = mdl.sum(link_vars[c_loc, b] * get_distance(c_loc, b) for c_loc in coffeeshop_locations for b in libraries)
mdl.minimize(total_distance)

In [47]:
print("# coffee shops locations = %d" % len(coffeeshop_locations))
print("# coffee shops           = %d" % nb_shops)

assert mdl.solve(), "!!! Solve or the model fails"

# coffee shops locations = 20
# coffee shops           = 5


# 3) Model Evaluation

In [48]:
total_distance = mdl.objective_value
open_coffeeshops = [c_loc for c_loc in coffeeshop_locations if coffeeshop_vars[c_loc].solution_value == 1]
not_coffeeshops = [c_loc for c_loc in coffeeshop_locations if c_loc not in open_coffeeshops]
edges = [(c_loc, b) for b in libraries for c_loc in coffeeshop_locations if int(link_vars[c_loc, b]) == 1]

print("Total distance = %g" % total_distance)
print("# coffee shops  = {0}".format(len(open_coffeeshops)))
for c in open_coffeeshops:
    print("new coffee shop: {0!s}".format(c))

Total distance = 32.8654
# coffee shops  = 5
new coffee shop: P_733 N. Kedzie Ave._25
new coffee shop: P_11071 S. Hoyne Ave._24
new coffee shop: P_1226 W. Ainslie St._69
new coffee shop: P_7455 W. Cornelia Ave._68
new coffee shop: P_2807 W. 55th St._66


In [51]:
# Display the solution
map_osm = folium.Map(location=[41.878, -87.629], zoom_start=11)
for i, coffeeshop in enumerate(open_coffeeshops):
    lt = coffeeshop.y
    lg = coffeeshop.x
    folium.Marker([lt, lg], icon=folium.Icon(color='red',icon='info-sign'), popup=f"Coffee Shop - {i}").add_to(map_osm)
    
for b in libraries:
    if b not in open_coffeeshops:
        lt = b.y
        lg = b.x
        folium.Marker([lt, lg]).add_to(map_osm)
    

for (c, b) in edges:
    coordinates = [[c.y, c.x], [b.y, b.x]]
    map_osm.add_child(folium.PolyLine(coordinates, color='#FF0000', weight=5))

map_osm